In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_sequence
import nltk
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score




#TRAIN LOADER


data = pd.read_csv(r'.\OLID_Tain_ATUSER_URL_EmojiRemoved_Pedro.txt', sep='\t', names=['id','sentence', 'label1','label2', 'label3'])  #Francesco esecution
data = data.drop(axis = 1, labels = ['id','label2','label3'])
data = data.drop(axis=0, index = 0 )

# checking how many pos and neg reviews
review_counts = data['label1'].value_counts()
print(f'Count of reviews by sentiment: {review_counts}')

print()
for index, row in data.iterrows():
    if row['label1'] == 'OFF':
        data.at[index, 'label1'] = 0
    else:
        data.at[index, 'label1'] = 1

# checking how many pos and neg reviews
review_counts = data['label1'].value_counts()
print(f'Count of reviews by sentiment: {review_counts}')
        


nltk.download('punkt')
data['tokens'] = data['sentence'].apply(word_tokenize)


# create a vocabulary and map tokens to indices
vocab = {word: idx for idx, word in enumerate(set(word for sentence in data['tokens'] for word in sentence), 1)}
data['indexed'] = data['tokens'].apply(lambda x: [vocab[word] for word in x])


# pad sequences to a maximum length
max_len = max(len(sentence) for sentence in data['indexed'])
data['padded'] = data['indexed'].apply(lambda x: x + [0]*(max_len - len(x)))

# convert to tensord and split dataset
features = torch.tensor(data['padded'].tolist())
labels = torch.tensor(data['label1'].tolist())


train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

train_data = TensorDataset(train_features, train_labels)
test_data = TensorDataset(test_features, test_labels)
train_loader = DataLoader(train_data, batch_size=10)
test_loader = DataLoader(test_data, batch_size=10)

Count of reviews by sentiment: label1
NOT    8840
OFF    4400
Name: count, dtype: int64

Count of reviews by sentiment: label1
1    8840
0    4400
Name: count, dtype: int64


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\39393\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
